Este cuaderno muestra cómo usar un modelo genérico de YOLOv10 para generar:

1. Un archivo de detecciones en formato MOT (`mot.txt`).
2. Un archivo de embeddings (`feats.npy`) y sus IDs (`ids.npy`) para evaluación de Re‑ID.

Puede procesar un vídeo o un directorio con imágenes (frames).


## 1. Instalación de dependencias

In [42]:
!pip install ultralytics opencv-python torch torchvision


## 2. Importar librerías y cargar modelo YOLOv10

In [43]:
# %%
from ultralytics import YOLO
import cv2
import numpy as np
import torch
from torchvision import transforms
from pathlib import Path

weights = 'yolov10n.pt'
# También puede sustituir por la ruta a su propio archivo .pt: e.g., 'path/to/yolov10.pt'
model = YOLO(weights)

# Verificar carga
print(f"Modelo YOLOv10 cargado: {weights}")


Modelo YOLOv10 cargado: yolov10n.pt



## 3. Configuración de la fuente de datos
- Si `use_video = True`, se leerá un vídeo.
- Si `use_video = False`, se leerá un directorio de imágenes.

In [44]:
# %%
# Configuración
datapath = r'C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4'  # Vídeo o carpeta de frames
use_video = True                # True: vídeo, False: directorio de imágenes
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

# Ruta de salida para MOT y ReID
det_file = output_dir / 'mot.txt'
feats_file = output_dir / 'feats_yolo10.npy'
ids_file  = output_dir / 'ids_yolo10.npy'
annotated_vid = output_dir / 'annotated.mp4'


## 4. Función de guardado de detecciones MOT

In [50]:
# %%

def save_mot_detections(detections, filepath):
    """
    Guarda detecciones en formato MOT:
    frame, id, x, y, w, h, score
    """
    with open(filepath, 'w') as f:
        for frame_id, obj_id, x, y, w, h, score in detections:
            f.write(f"{frame_id},{obj_id},{x:.2f},{y:.2f},{w:.2f},{h:.2f},{score:.4f},-1,-1,-1\n")


## 5. Extracción de embeddings (ReID)

Se asume un extractor simple; debe adaptarse a su modelo de ReID real.


In [46]:


# %%
reid_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

class DummyReID(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = torch.nn.AdaptiveAvgPool2d((1,1))
    def forward(self, x):
        x = self.pool(x)
        return x.view(x.size(0), -1)

reid_model = DummyReID().eval()

# Variables para acumular resultados
detections_all = []  # [(frame, id, x, y, w, h, score), ...]
feats_list     = []  # [embedding,...]
ids_list       = []  # [obj_id,...]



## 6. Procesamiento de frames

In [47]:
# %%
frame_id = 0

# Fuente: vídeo o directorio de imágenes
source = datapath if use_video else str(Path(datapath) / '*.jpg')

# Procesamiento frame a frame con seguimiento
for result in model.track(source=source, stream=True):
    frame_id += 1
    boxes = result.boxes  # Boxes con .xyxy (1x4), .conf, .id
    frame = result.orig_img
    for box in boxes:
        # Obtener coordenadas unpackeables
        coords = box.xyxy[0].tolist()  # [[x1,y1,x2,y2]] -> [x1,y1,x2,y2]
        x1, y1, x2, y2 = coords
        conf = float(box.conf)
        obj_id = int(box.id)
        w, h = x2 - x1, y2 - y1

        # Guardar detección con ID
        detections_all.append((frame_id, obj_id, x1, y1, w, h, conf))

        # Extraer crop y embedding
        crop = frame[int(y1):int(y1+h), int(x1):int(x1+w)]
        if crop.size:
            inp = reid_transform(crop).unsqueeze(0)
            with torch.no_grad():
                feat = reid_model(inp)
            feats_list.append(feat.cpu().numpy().squeeze())
            ids_list.append(obj_id)




video 1/1 (frame 1/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 10 persons, 113.3ms
video 1/1 (frame 2/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 10 persons, 103.2ms
video 1/1 (frame 3/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 10 persons, 88.7ms
video 1/1 (frame 4/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 10 persons, 83.6ms
video 1/1 (frame 5/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 11 persons, 82.9ms
video 1/1 (frame 6/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 11 persons, 81.2ms
video 1/1 (frame 7/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 11 persons, 95.2ms
video 1/1 (frame 8/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 12 persons, 84.2ms
video 1/1 (frame 9/1578) C:\Users\Soriano\OneDrive\Documentos\Deep-EIoU\video2.mp4: 384x640 12 persons, 134.4


## 7. Guardar resultados

In [51]:
# %%
# Guardar detecciones MOT
save_mot_detections(detections_all, det_file)
print(f"Detecciones MOT guardadas en: {det_file}")

# Guardar embeddings y IDs
np.save(feats_file, np.vstack(feats_list))
np.save(ids_file, np.array(ids_list))
print(f"Embeddings guardados en: {feats_file}")
print(f"IDs guardados en: {ids_file}")

Detecciones MOT guardadas en: outputs\mot.txt
Embeddings guardados en: outputs\feats_yolo10.npy
IDs guardados en: outputs\ids_yolo10.npy


In [52]:

# %%
# Releer vídeo o frames y dibujar cajas+IDs
if use_video:
    cap = cv2.VideoCapture(datapath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    writer = cv2.VideoWriter(str(annotated_vid), cv2.VideoWriter_fourcc(*'mp4v'), fps, (width,height))
    frame_idx = 0
    det_dict = {}  # {frame: [(id,x1,y1,x2,y2)]}
    for f,oid,x,y,w,h,conf in detections_all:
        det_dict.setdefault(f,[]).append((oid,int(x),int(y),int(x+w),int(y+h)))
    while True:
        ret, frame = cap.read()
        if not ret: break
        frame_idx += 1
        for oid,x1,y1,x2,y2 in det_dict.get(frame_idx,[]):
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.putText(frame,f"ID:{oid}",(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),2)
        writer.write(frame)
    cap.release(); writer.release()
    print(f"Vídeo anotado guardado en: {annotated_vid}")
else:
    print("Anotación para directorio de imágenes no implementada.")

Vídeo anotado guardado en: outputs\annotated.mp4
